In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import torch
import torch.nn
import torch.optim
import torch.utils.data
import h5py
from os.path import exists
import time
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, roc_auc_score


# Importing LearningCutsUtils
from LearningCutsUtils import OneToOneLinear, EfficiencyScanNetwork
from LearningCutsUtils import loss_fn, effic_loss_fn, lossvars
import LearningCutsUtils.Utils as LCU

In [ ]:
# Prep Data using Pandas
x_sig_data=None
y_sig_data=None

x_bkg_data=None
y_bkg_data=None

num_sig_events=0
num_bkg_events=0

# Need to copy the last branch, Rll, so that we have a total of 5 branches
branches=(
    'lep1MT_Met',
    'lep2MT_Met',
    'met_Et',
    'Rll')

# open signal
mass=200
split=30
filepath='/data/mhance/SUSY/Compressed/'
filebase='SusySkimSlep_v0.2_SlepSignals__'
filename='MGPy8EG_A14N23LO_SlepSlep_dir_2L2MET75_%dp0_%dp0_NoSys' % (mass,mass-split)
filesuff='.hf5'
fullname=filepath+filebase+filename+filesuff

new_branches = ('lep1MT_Met',
    'lep2MT_Met',
    'met_Et',
    'Rll',
    'Rll_lt')
# Makes data frame for signal events
num_sig_events = len(np.array(h5py.File(fullname)['MGPy8EG_A14N23LO_SlepSlep_dir_2L2MET75_200p0_170p0_NoSys'])["nJet30"])
df1 = pd.DataFrame(np.array(h5py.File(fullname)['MGPy8EG_A14N23LO_SlepSlep_dir_2L2MET75_200p0_170p0_NoSys']['lep1MT_Met', 'lep2MT_Met', 'met_Et','Rll']), columns=branches)
df1["Rll_lt"] = df1.iloc[:, -1]
# Makes ndarray for xsignal using data frame
x_sig_data = df1[:].values
y_sig_data=np.ones(num_sig_events)

print(fullname)
print("Extracted %7d signal events" % num_sig_events)

fullname=filepath+"SusySkimSlep_v0.2_diboson2L__diboson2L_NoSys"+filesuff
df2 = pd.DataFrame(np.array(h5py.File(fullname)['diboson2L_NoSys']['lep1MT_Met', 'lep2MT_Met', 'met_Et','Rll']), columns=branches)
df2["Rll_lt"] = df2.iloc[:, -1]

num_events=0

num_bkg_events=len(np.array(h5py.File(fullname)['diboson2L_NoSys']["nJet30"]))
x_bkg_data = df2[:].values
y_bkg_data=np.zeros(num_bkg_events)
print("Extracted %7d background events" % num_bkg_events)

In [ ]:
x_data=None
y_data=None
if num_bkg_events>num_sig_events:
    x_data = np.concatenate((x_sig_data,x_bkg_data[:num_sig_events]))
    y_data = np.concatenate((y_sig_data,y_bkg_data[:num_sig_events]))
else:
    x_data = np.concatenate((x_sig_data[:num_bkg_events],x_bkg_data))
    y_data = np.concatenate((y_sig_data[:num_bkg_events],y_bkg_data))

print(y_sig_data)
print(y_data)

In [ ]:
# we read in the data as fields with a custom format, which is useful for keeping track of what's what, but 
# ML libraries wants everything as tuples of floats.  
#x_data=[tuple(float(i) if np.isfinite(float(i)) else 0 for i in j) for j in x_data]
x_data=[tuple(float(i) for i in j) for j in x_data]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, 
                                                    test_size=int(0.1*len(x_data)), 
                                                    random_state=123)

In [ ]:
x_train_before_scaling={}
for b in new_branches:
    x_train_before_scaling[b]=[event[new_branches.index(b)] for event in x_train]

In [ ]:
fig = plt.figure(figsize=(20,30))
fig.tight_layout()
for b in range(len(new_branches)):
    ax=fig.add_subplot(10,5,1+b)
    plt.subplots_adjust(hspace=0.3,wspace=0.5)
    ax.hist(x_train_before_scaling[new_branches[b]])
    ax.set_xlabel(new_branches[b])
    ax.set_ylabel("Events/Bin")

In [ ]:
N=20000 # number of points
m=5 # dimensions

In [ ]:
# now scale based on the training data:
sc = StandardScaler()

x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
x_train_after_scaling={}
for b in new_branches:
    x_train_after_scaling[b]=[event[new_branches.index(b)] for event in x_train]

In [ ]:
fig = plt.figure(figsize=(20,30))
fig.tight_layout()
for b in range(len(new_branches)):
    ax=fig.add_subplot(10,5,1+b)
    plt.subplots_adjust(hspace=0.3,wspace=0.5)
    ax.hist(x_train_after_scaling[new_branches[b]])
    ax.set_xlabel(new_branches[b])
    ax.set_ylabel("Events/Bin")

In [ ]:
x_sig_data = np.transpose(x_sig_data)

x_bkg_data = np.transpose(x_bkg_data)

In [ ]:
ig = plt.figure(figsize=(20,5))
fig.tight_layout()
nbins=50

for b in range(len(new_branches)):
    fig = plt.figure(figsize=(20,5))
    ax=fig.add_subplot(2,5,1+b)
    plt.subplots_adjust(hspace=0.3,wspace=0.5)
    plt.yscale('log')
    ax.hist(x_sig_data[b],nbins,density=True,histtype='stepfilled',alpha=0.5,color='red')
    ax.hist(x_bkg_data[b],nbins,density=True,histtype='stepfilled',alpha=0.5,color='blue')
    ax.set_xlabel(f"Feature {new_branches[b]}")
    ax.set_ylabel("Events/Bin")

In [ ]:
torch.manual_seed(123)
net = torch.nn.Sequential(
    torch.nn.Linear(len(branches), 20),
    torch.nn.Tanh(),
    torch.nn.Linear(20, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50, 20),
    torch.nn.ReLU(),
    torch.nn.Linear(20, 1)
)
torch.save(net.state_dict(), 'net.pth')
#loss_fn = torch.nn.MSELoss()
loss_fn = torch.nn.BCEWithLogitsLoss()
#optimizer = torch.optim.SGD(net.parameters(), lr=0.05, momentum=0.9)
optimizer = torch.optim.Adam(net.parameters(), lr=0.02)

In [ ]:
x_train_tensor=torch.tensor(x_train,dtype=torch.float)
y_train_tensor=torch.tensor(y_train,dtype=torch.float)
y_train_tensor=y_train_tensor.unsqueeze(1)

print(x_train_tensor.shape)

x_test_tensor=torch.tensor(x_test,dtype=torch.float)
y_test_tensor=torch.tensor(y_test,dtype=torch.float)
y_test_tensor=y_test_tensor.unsqueeze(1)

In [ ]:
import importlib
import LearningCutsUtils.LearningCutsUtils
## needed if we change LearningCutsUtils and want to avoid reloading the kernel to see the effects
importlib.reload(LearningCutsUtils.LearningCutsUtils)
import LearningCutsUtils.LearningCutsUtils as LCU
from LearningCutsUtils import loss_fn
from LearningCutsUtils import effic_loss_fn


N=20000 # number of points
m=5 # dimensions

In [ ]:
gt=-1.
lt=1.

cuts_gt_lt = [lt, lt, gt, gt, lt]

In [ ]:
def function_to_loop(activation_input_scale_factor=15., learning_rate=0.1, batch_size=int(len(y_train)/20.), epochs = 50, alpha = 10., beta=0.1, gamma=1e-2, target_efficiency=0.8):
    net = OneToOneLinear(m,activation_input_scale_factor,cuts_gt_lt)
    torch.save(net.state_dict(), 'net_learningbiases.pth')
    #optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
    {n: theta.shape for n, theta in net.named_parameters()}
    losses = []
    losses_test = []

    net.load_state_dict(torch.load('net_learningbiases.pth',weights_only=True))

    xy_train = torch.utils.data.TensorDataset(x_train_tensor.float(),y_train_tensor)
    loader = torch.utils.data.DataLoader(xy_train, batch_size=batch_size, shuffle=True)
    debug=False

    for epoch in range(epochs):
        net.train()
        start_time = time.time()
        for x_batch, y_batch in loader:
            y_pred = net(x_batch)
            optimizer.zero_grad()
            loss = loss_fn(y_pred, y_batch.squeeze(1), m, net, target_efficiency, alpha, beta, gamma)
            loss.totalloss().backward()
            optimizer.step()
        losses.append(loss)
        net.eval() # configure the model for evaluation (testing)
        y_pred = net(x_test_tensor)
        test_loss =loss_fn(y_pred, y_test_tensor.squeeze(1), m, net, target_efficiency, alpha, beta, gamma)
        losses_test.append(test_loss)
        end_time=time.time()
        elapsed_time = end_time - start_time
        bias=net.bias[0]
        weight=net.weight[0]
        #weight={weight:4.1e}, bias={bias:4.1e}, 
        print(f"Completed epoch {epoch:2d} in {elapsed_time:4.1f}s, Train loss={loss.totalloss().data:4.1e}, Test loss={test_loss.totalloss().data:4.1e}, cut={-bias/weight:4.1e}, sig_eff={100*test_loss.signaleffic:4.1f}%, bkg_eff={100*test_loss.backgreffic:6.3f}%")
    net.eval() # configure the model for evaluation (testing)
    y_pred_test = net(x_test_tensor).detach().cpu()
    y_pred_train= net(x_train_tensor).detach().cpu()
    LearningCutsUtils.LearningCutsUtils.make_ROC_curve(y_test, y_pred_test)
    return y_pred_test, y_pred_train

In [ ]:
y_pred_test, y_pred_train = function_to_loop(activation_input_scale_factor=15., learning_rate=0.5, batch_size=int(len(y_train)/1.), epochs = 150, alpha = 10., beta=0.1, gamma=10**(-5), target_efficiency=0.8)

In [ ]:
LCU.plot_classifier_output(y_train, y_pred_train, y_test, y_pred_test, nbins=20, range=(0,1))